In [1]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory
)
from langchain_core.runnables.history import RunnableWithMessageHistory


In [2]:
store = {}

In [3]:
def get_session_history(session_id: str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [3]:
from langchain_groq import ChatGroq
import os

model = ChatGroq(api_key=os.getenv('GROQ_API_KEY'),model='mixtral-8x7B-32768')

In [140]:
from langchain_core.messages import SystemMessage,HumanMessage
response = model.invoke([
    SystemMessage(content='You are a calculator and you only return a single number in output'),
    HumanMessage(content=('What is (5 plus 7)'))])

In [141]:
print(response)

content='12' response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 31, 'total_tokens': 35, 'completion_time': 0.00625, 'prompt_time': 0.002980294, 'queue_time': None, 'total_time': 0.009230294}, 'model_name': 'mixtral-8x7B-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None} id='run-921e7e86-d84b-4f0c-a441-e7eb4464a3e6-0' usage_metadata={'input_tokens': 31, 'output_tokens': 4, 'total_tokens': 35}


In [5]:
with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [6]:
config = {'configurable':{'session_id':'abc2'}}

In [13]:
from typing import List


from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, AIMessage
from langchain_core.pydantic_v1 import BaseModel, Field



class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

# Here we use a global variable to store the chat message history.
# This will make it easier to inspect it to see the underlying results.
store = {}

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]


history = get_by_session_id("1")
history.add_message(AIMessage(content="hello"))
print(store)  # noqa: T201

{'1': InMemoryHistory(messages=[AIMessage(content='hello')])}


In [15]:
history.add_message(AIMessage(content='hello im akshay'))

In [17]:
from langchain_core.prompts import MessagesPlaceholder,ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory

In [19]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an assistant who's good at {ability}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

In [20]:
chain = prompt | model

In [21]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_by_session_id,
    input_messages_key='question',
    history_messages_key='history'
)

In [22]:
print(chain_with_history.invoke(
    {'ability':'math','question':'What does cosine mean?'},
    config = {'configurable':{'session_id':'foo'}}
))

content='The cosine function, often abbreviated as cos, is a fundamental concept in trigonometry, a branch of mathematics that deals with the relationships between the angles and sides of triangles. The cosine of an angle in a right triangle is defined as the length of the adjacent side divided by the length of the hypotenuse.\n\nIn the context of a unit circle (a circle with a radius of 1), the cosine of an angle is the x-coordinate of the point on the circle that corresponds to the angle.\n\nCosine is a periodic function, meaning that it repeats its values in regular intervals. The period of the cosine function is 2π (or 360 degrees), which means that the values of cosine repeat every 2π radians (or 360 degrees).\n\nThe cosine function is often used in various applications, including physics, engineering, computer graphics, and many other fields, to model periodic phenomena or to solve problems involving right triangles.' response_metadata={'token_usage': {'completion_tokens': 219, '

In [131]:
print(store)

{'1': InMemoryHistory(messages=[AIMessage(content='hello'), AIMessage(content='hello im akshay'), AIMessage(content='hello im akshay')]), 'foo': InMemoryHistory(messages=[HumanMessage(content='What does cosine mean?'), AIMessage(content='The cosine function, often abbreviated as cos, is a fundamental concept in trigonometry, a branch of mathematics that deals with the relationships between the angles and sides of triangles. The cosine of an angle in a right triangle is defined as the length of the adjacent side divided by the length of the hypotenuse.\n\nIn the context of a unit circle (a circle with a radius of 1), the cosine of an angle is the x-coordinate of the point on the circle that corresponds to the angle.\n\nCosine is a periodic function, meaning that it repeats its values in regular intervals. The period of the cosine function is 2π (or 360 degrees), which means that the values of cosine repeat every 2π radians (or 360 degrees).\n\nThe cosine function is often used in variou

In [24]:
print(chain_with_history.invoke(  # noqa: T201
    {"ability": "math", "question": "What's its inverse"},
    config={"configurable": {"session_id": "foo"}}
))


content='The inverse of the cosine function, often denoted as arccos or cos^-1, is a function that takes a value between -1 and 1 and returns the angle for which the cosine is that value. In other words, if you input a number x between -1 and 1 into the arccos function, it will output the angle θ such that cos(θ) = x.\n\nThe arccos function is defined for all real numbers x such that -1 ≤ x ≤ 1, and its output is an angle in radians. The output of the arccos function is always between 0 and π radians (or 180 degrees and 360 degrees, respectively) for real values of x.\n\nIt is important to note that the inverse cosine function is a many-to-one function, meaning that there are multiple angles that have the same cosine value. For example, both 0 and 2π radians have a cosine value of 1. To avoid ambiguity, most calculators and software assume that the output of the arccos function is the angle in the range of 0 to π radians (or 180 degrees to 360 degrees) when the input is a real number b

In [132]:
print(store)

{'1': InMemoryHistory(messages=[AIMessage(content='hello'), AIMessage(content='hello im akshay'), AIMessage(content='hello im akshay')]), 'foo': InMemoryHistory(messages=[HumanMessage(content='What does cosine mean?'), AIMessage(content='The cosine function, often abbreviated as cos, is a fundamental concept in trigonometry, a branch of mathematics that deals with the relationships between the angles and sides of triangles. The cosine of an angle in a right triangle is defined as the length of the adjacent side divided by the length of the hypotenuse.\n\nIn the context of a unit circle (a circle with a radius of 1), the cosine of an angle is the x-coordinate of the point on the circle that corresponds to the angle.\n\nCosine is a periodic function, meaning that it repeats its values in regular intervals. The period of the cosine function is 2π (or 360 degrees), which means that the values of cosine repeat every 2π radians (or 360 degrees).\n\nThe cosine function is often used in variou

In [1]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory
)
from langchain_core.runnables.history import RunnableWithMessageHistory

In [4]:
store = {}
def get_session_history(session_id: str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model,get_session_history)